# Subgraph Withholding Attack EDA

In [1]:
# standard imports
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_rows', 15)

In [2]:

# function used to query each type of event from The Graph
def query_theGraph_epoch(raw_query, field_name, url, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}" 
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:endBlock, where:{id:"1"})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:endBlock, where: {endBlock_gt: ' + str(indexer) + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['endBlock']
            print(indexer)
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99 and n > 1:
            break
        
    return pd.DataFrame(records)

In [3]:
#get subgraphs data
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

epoch_query = '''query{
	epoches(){
    id
    signalledTokens
    stakeDeposited
    endBlock
  }
}
'''

field_name = 'epoches'

epoch_df = query_theGraph_epoch(epoch_query, field_name, url, False)

11460060
12131306
12795906
13460506
14125106
14789706
14796352


In [4]:
epoch_df

,id,signalledTokens,stakeDeposited,endBlock
0,1,0,10000000000000000000,11460060
1,3,0,2137573000000000000000000,11473352
2,4,244919450103834483981271,72326775000000000000000000,11479998
3,5,0,48119513876353513935471811,11486644
4,6,0,58229882095246030700280898,11493290
...,...,...,...,...
497,499,57210954499988596409699,1209426397765591050149705,14769768
498,500,32508847055007768426375,261964898304049647857916,14776414
499,501,297000000000000000000,1041607276511850509529,14783060
500,502,23023426149394278110006,328056990049068425245660,14789706


In [5]:
epoch_df['signalledTokens'] = epoch_df['signalledTokens'].astype(float)
epoch_df['signalledTokens'] = epoch_df['signalledTokens']/10e18


In [6]:
epoch_df['stakeDeposited'] = epoch_df['stakeDeposited'].astype(float)
epoch_df['stakeDeposited'] = epoch_df['stakeDeposited']/10e18

In [7]:
epoch_df

,id,signalledTokens,stakeDeposited,endBlock
0,1,0.000000,1.000000e+00,11460060
1,3,0.000000,2.137573e+05,11473352
2,4,24491.945010,7.232678e+06,11479998
3,5,0.000000,4.811951e+06,11486644
4,6,0.000000,5.822988e+06,11493290
...,...,...,...,...
497,499,5721.095450,1.209426e+05,14769768
498,500,3250.884706,2.619649e+04,14776414
499,501,29.700000,1.041607e+02,14783060
500,502,2302.342615,3.280570e+04,14789706


In [8]:
epoch_df['cumulative_tokens'] = epoch_df['stakeDeposited'] + epoch_df['signalledTokens']
epoch_df['total_supply'] = epoch_df['cumulative_tokens'].cumsum()
epoch_df

,id,signalledTokens,stakeDeposited,endBlock,cumulative_tokens,total_supply
0,1,0.000000,1.000000e+00,11460060,1.000000e+00,1.000000e+00
1,3,0.000000,2.137573e+05,11473352,2.137573e+05,2.137583e+05
2,4,24491.945010,7.232678e+06,11479998,7.257169e+06,7.470928e+06
3,5,0.000000,4.811951e+06,11486644,4.811951e+06,1.228288e+07
4,6,0.000000,5.822988e+06,11493290,5.822988e+06,1.810587e+07
...,...,...,...,...,...,...
497,499,5721.095450,1.209426e+05,14769768,1.266637e+05,1.359968e+08
498,500,3250.884706,2.619649e+04,14776414,2.944737e+04,1.360263e+08
499,501,29.700000,1.041607e+02,14783060,1.338607e+02,1.360264e+08
500,502,2302.342615,3.280570e+04,14789706,3.510804e+04,1.360615e+08


In [9]:
epoch_df['signalled_total'] = epoch_df['signalledTokens'].cumsum()
epoch_df

,id,signalledTokens,stakeDeposited,endBlock,cumulative_tokens,total_supply,signalled_total
0,1,0.000000,1.000000e+00,11460060,1.000000e+00,1.000000e+00,0.000000e+00
1,3,0.000000,2.137573e+05,11473352,2.137573e+05,2.137583e+05,0.000000e+00
2,4,24491.945010,7.232678e+06,11479998,7.257169e+06,7.470928e+06,2.449195e+04
3,5,0.000000,4.811951e+06,11486644,4.811951e+06,1.228288e+07,2.449195e+04
4,6,0.000000,5.822988e+06,11493290,5.822988e+06,1.810587e+07,2.449195e+04
...,...,...,...,...,...,...,...
497,499,5721.095450,1.209426e+05,14769768,1.266637e+05,1.359968e+08,2.088015e+06
498,500,3250.884706,2.619649e+04,14776414,2.944737e+04,1.360263e+08,2.091266e+06
499,501,29.700000,1.041607e+02,14783060,1.338607e+02,1.360264e+08,2.091296e+06
500,502,2302.342615,3.280570e+04,14789706,3.510804e+04,1.360615e+08,2.093598e+06


In [10]:
epoch_df.rename({'id': 'epoch'}, axis = 1, inplace=True)
epoch_df

,epoch,signalledTokens,stakeDeposited,endBlock,cumulative_tokens,total_supply,signalled_total
0,1,0.000000,1.000000e+00,11460060,1.000000e+00,1.000000e+00,0.000000e+00
1,3,0.000000,2.137573e+05,11473352,2.137573e+05,2.137583e+05,0.000000e+00
2,4,24491.945010,7.232678e+06,11479998,7.257169e+06,7.470928e+06,2.449195e+04
3,5,0.000000,4.811951e+06,11486644,4.811951e+06,1.228288e+07,2.449195e+04
4,6,0.000000,5.822988e+06,11493290,5.822988e+06,1.810587e+07,2.449195e+04
...,...,...,...,...,...,...,...
497,499,5721.095450,1.209426e+05,14769768,1.266637e+05,1.359968e+08,2.088015e+06
498,500,3250.884706,2.619649e+04,14776414,2.944737e+04,1.360263e+08,2.091266e+06
499,501,29.700000,1.041607e+02,14783060,1.338607e+02,1.360264e+08,2.091296e+06
500,502,2302.342615,3.280570e+04,14789706,3.510804e+04,1.360615e+08,2.093598e+06


In [27]:
epoch_df.to_csv('./total_supply.csv')

In [11]:
curation_df = pd.read_csv('../curation_events/curation_events.csv')
curation_df.drop('Unnamed: 0', axis = 1, inplace = True)
curation_df

,id,blockNumber,signer,type,nameSignal,versionSignal,tokens,subgraphId,subgraphDeploymentID,subgraphversion,subgraphname
0,0x4c1060853752b5221b1e27c2264034d0bd5fd86c5267...,11474121,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,MintNSignal,279284800875378823397,279284800875378823397,80000000000000000000000,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0,PoolTogether
1,0xea7dbe2f6164530a53ac8cf9d99b46993f44d8ca8258...,11476852,0x0f3cf18b37c55f5a319d6cae8d8ba114cb094af0,MintNSignal,627949250367270370,627949250367270370,360151469700892136118,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0,PoolTogether
2,0xfbdb37e510284ebbeb45fbfdf28191219faf84bc31bf...,11476875,0x0f3cf18b37c55f5a319d6cae8d8ba114cb094af0,MintNSignal,125503147008134479971,125503147008134479971,88216309708626528177242,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0,PoolTogether
3,0xfbda17a348fb979ad36e27445d656b23458a938fadd2...,11476931,0xb6916933aa0bac4b6d09260719c4055363642745,MintNSignal,89477475325111327822,89477475325111327822,82622974825605383769993,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0,PoolTogether
4,0x12925f181018abc41ef4fd32c6e95dab9ff85a6a8337...,11692858,0x4f4538e2553c61bce747f73c5fe8133d4b383dda,MintNSignal,3085781289271725739,3085781289271725739,2000000000000000000000,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0,PoolTogether
...,...,...,...,...,...,...,...,...,...,...,...
7506,0xdc0c94cb7576fd82b40d94a5f18e46eab340d2070fd6...,14704848,0x79bdd2c976685f500a6e0f601b496acff20b56a7,MintNSignal,99498743710661995473,99498743710661995473,10000000000000000000000,DfD1tZSmDtjCGC2LeYEQbVzj9j8kNqKAQEsYL27Vg6Sw,0x83fc1011f4ab60f6c9e693785c731604affbefbeeccb...,0,Connext Network
7507,0xcb7f61c448b0c42f509dc96047522ee702f1aee07e42...,14705527,0xf6c0366cc3454c259b2d4864fa6bb3874e2590a5,MintNSignal,65671660990071637813,65671660990071637813,28598662055154772118285,7QP7oCLbEAjejkp7wSLTD1zbRMSiDydAmALksBB5E6i1,0xc0bb19a3321ddd3ee6a956bdea9c98e5954feb57d4c2...,0,Loopring zkRollup
7508,0x62d394c215c6ce5725ea35612542c67837b3511b8221...,14707168,0x0b5cf9dc29003890f791553e1e9b447339e35df3,MintNSignal,1407124727947028866,1407124727947028866,2000000000000000000,5gyzN7gutVUnen2mi7HEFdgm9q2vkcaaYd4BBE9GA9WT,0xb952f95b17a00987492361e8aecc1e5e4a7c508b8d5e...,0,cb-extn0502mn
7509,0x4dd275879e57dc2afc1cefd5ff39af9cea1a5c783cdc...,14707641,0x0b5cf9dc29003890f791553e1e9b447339e35df3,MintNSignal,1407124727947028866,1407124727947028866,2000000000000000000,9yhUteMHTNCqydn2gBHcNJ7wERu1KN6H6NJvB9nqi3Fq,0x22df70e1bcbab8f5660ecb9421637a61d7660885991c...,0,cb-ext0502mn2


In [13]:
curation_df['epoch'] = (curation_df['blockNumber'] - 11446768) / 6646
curation_df['epoch'] = curation_df['epoch'].astype(int)

In [14]:
curation_df['tokens'] = curation_df['tokens'].astype(float)
curation_df['tokens'] = curation_df['tokens']/10e18

In [23]:
curation_by_subgraph = {}
for i, j in curation_df.iterrows():
    if j['subgraphId'] not in curation_by_subgraph:
        curation_by_subgraph[j['subgraphname']] = {}
        curation_by_subgraph[j['subgraphname']][j['epoch']] = j['tokens']
    else:
        curation_by_subgraph[j['subgraphname']][j['epoch']] = j['tokens']
curation_by_subgraph_df = pd.DataFrame(curation_by_subgraph)
curation_by_subgraph_df

,PoolTogether,RAI Mainnet,mStable Protocol,UMA Mainnet Voting,Opyn Gamma Mainnet,DODOEX V2,Omen,Audius Network Mainnet,Enzyme Finance,Livepeer,...,Swapr Mainnet,gnosis-safe-mainnet,DODOEX V2 Polygon,UpstreamCollectives,Scab Shop,chainlink-mainnet-ocr,Ampleforth DAO,Connext Network,cb-extn0502mn,cb-ext0502mn2
201,68.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433,NaN,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442,NaN,NaN,1584.0,NaN,NaN,NaN,NaN,199.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
489,NaN,NaN,NaN,110.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11.9,NaN,NaN,NaN,NaN,NaN,NaN
457,NaN,NaN,NaN,NaN,0.001,0.005,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
missing_epochs = pd.Index(range(492)).difference(curation_by_subgraph_df.index)
missing_epochs_df = pd.DataFrame(index=missing_epochs, columns=curation_by_subgraph_df.columns).fillna(0.0)
full_df = pd.concat([curation_by_subgraph_df, missing_epochs_df])

In [25]:
full_df = full_df.sort_index()
full_df = full_df.fillna(0)
full_df

,PoolTogether,RAI Mainnet,mStable Protocol,UMA Mainnet Voting,Opyn Gamma Mainnet,DODOEX V2,Omen,Audius Network Mainnet,Enzyme Finance,Livepeer,...,Swapr Mainnet,gnosis-safe-mainnet,DODOEX V2 Polygon,UpstreamCollectives,Scab Shop,chainlink-mainnet-ocr,Ampleforth DAO,Connext Network,cb-extn0502mn,cb-ext0502mn2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489,0.0,0.0,0.0,110.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,11.9,0.0,0.0,0.0,0.0,0.0,0.0
490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,50.0,0.0,0.0,0.0,0.0,1000.0,0.2,0.2


In [26]:
for i in full_df.columns:
    full_df[i] = full_df[i].cumsum()
full_df

,PoolTogether,RAI Mainnet,mStable Protocol,UMA Mainnet Voting,Opyn Gamma Mainnet,DODOEX V2,Omen,Audius Network Mainnet,Enzyme Finance,Livepeer,...,Swapr Mainnet,gnosis-safe-mainnet,DODOEX V2 Polygon,UpstreamCollectives,Scab Shop,chainlink-mainnet-ocr,Ampleforth DAO,Connext Network,cb-extn0502mn,cb-ext0502mn2
0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,68.5,500.0,1584.0,0.0,0.001,0.005,30.0,199.7,13.1,248.17,...,1000.0,140.0,0.0,0.0,100.0,953.639055,1500.0,0.0,0.0,0.0
488,68.5,500.0,1584.0,0.0,0.001,0.005,30.0,199.7,13.1,248.17,...,1000.0,140.0,0.0,0.0,100.0,953.639055,1500.0,0.0,0.0,0.0
489,68.5,500.0,1584.0,110.0,0.001,0.005,30.0,199.7,13.1,248.17,...,1000.0,140.0,0.0,11.9,100.0,953.639055,1500.0,0.0,0.0,0.0
490,68.5,500.0,1584.0,110.0,0.001,0.005,30.0,199.7,13.1,248.17,...,1000.0,140.0,50.0,11.9,100.0,953.639055,1500.0,1000.0,0.2,0.2


In [28]:
full_df.to_csv('./curation_subgraph.csv')